In [11]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataset_tools import *

In [12]:
def process_batch(out_dir,imgs,masks,w,h,min_r,min_roi):
    cols = imgs.shape[2]//w
    rows = imgs.shape[1]//h
    imgs = imgs[:,:h*rows,:w*cols,:]
    masks = masks[:,:h*rows,:w*cols]
    imgs = np.transpose(imgs.reshape((-1,rows,h,cols,w,3)),(1,3,0,2,4,5)).reshape(cols*rows,len(imgs),h,w,3)
    masks = np.transpose(masks.reshape((-1,rows,h,cols,w)),(1,3,0,2,4)).reshape(cols*rows,len(masks),h,w)
    motion = np.zeros_like(masks)
    motion[masks > 240] = 1
    motion = motion.mean((2,3))

    roi = np.zeros_like(masks)
    roi[(masks > 240) | (masks < 10)] = 1
    roi = roi.mean((2,3))
    iter = 0
    for i in range(len(masks)):
        if(motion[i].mean() >= min_r and roi[i].mean() >= min_roi):
            cv2.imwrite(out_dir+'/%d_input.jpg'%(iter),ties_to_image(imgs[i]))
            cv2.imwrite(out_dir+'/%d_mask.png'%(iter),ties_to_image(masks[i]))
            iter+=1
    return imgs.reshape((-1,h,w,3))[motion.flatten()>=.97],iter


def save_motion(out_dir,motion_ties):
    if not (motion_ties is None):
        for i in range(int(np.ceil(float(len(motion_ties))/50.))):
            im = ties_to_image(motion_ties[i*50:min((i+1)*50,len(motion_ties))],5,10)
            cv2.imwrite(out_dir+'/%d_motion.jpg'%(i),im)
        return len(motion_ties)
    else:
        return 0

In [13]:
def generate_ties(dataset,out_dir,length,tie_size,min_r=0,min_roi = 0.5):
    for video_num,(in_dir,out_dir) in enumerate(iterate_folders(dataset,out_dir)):
        ties_num = 0
        motion_ties = None
#         f = [int(ii) for ii in (open(in_dir+'/temporalROI.txt').read()).split(' ')]
#         end = f[-1]
#         for i in range(f[0],f[1]):
#             m = cv2.imread(in_dir+'/groundtruth/gt%06d.png'%(i))
#             if(m[((m > 10)&(m < 240))].size > 0.9*m.size):
#                 end = i
#                 break
#         batches = (end-f[0]-1)//length
#         print batches
        for batch_num,(imgs,masks) in enumerate(iterate_bathced(in_dir,length)):
                
#             if(batch_num == batches/2):
#                 m = save_motion(out_dir,motion_ties)
#                 print out_dir+ ' %d ties=%d motion=%d'%(batch_num,ties_num,m)
#                 if(m == 0):
#                     break
#                 out_dir = 'test/'+out_dir
#                 make_path(out_dir)
#                 motion_ties = None
#                 ties_num = 0
                
#             if(batch_num >= batches):
#                 break
            m_ties,i = process_batch(out_dir,imgs,masks,tie_size,tie_size,min_r,min_roi)
            ties_num += i
            if(len(m_ties) > 0):
                if not(motion_ties is None):
                    motion_ties = np.concatenate((motion_ties,m_ties))
                else:
                    motion_ties = m_ties
            
                 
        print out_dir+ ' %d ties=%d motion=%d'%(batch_num,ties_num,save_motion(out_dir,motion_ties))

In [14]:
generate_ties('/media/hpc2_storage/nvaulin/dataset','/media/hpc2_storage/nvaulin/ties32',256,32,-1.,0.95)

/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence2 14 ties=1260 motion=0
/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence3 4 ties=652 motion=1524
/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence1 9 ties=1623 motion=4
/media/hpc2_storage/nvaulin/ties32/thermal/lakeSide 20 ties=1309 motion=23
/media/hpc2_storage/nvaulin/ties32/thermal/park 0 ties=99 motion=30
/media/hpc2_storage/nvaulin/ties32/thermal/diningRoom 10 ties=675 motion=4509
/media/hpc2_storage/nvaulin/ties32/thermal/library 15 ties=858 motion=19177
/media/hpc2_storage/nvaulin/ties32/PTZ/zoomInZoomOut 1 ties=68 motion=0
/media/hpc2_storage/nvaulin/ties32/PTZ/twoPositionPTZCam 4 ties=60 motion=125
/media/hpc2_storage/nvaulin/ties32/PTZ/intermittentPan 7 ties=370 motion=454
/media/hpc2_storage/nvaulin/ties32/dynamicBackground/overpass 6 ties=434 motion=294
/media/hpc2_storage/nvaulin/ties32/dynamicBackground/fall 10 ties=3627 motion=11867
/media/hpc2_storage/nvaulin/ties32/dynamicBackground/canoe 0 ties=51

In [15]:
generate_ties('/media/hpc2_storage/nvaulin/test','/media/hpc2_storage/nvaulin/test_ties32',256,32,-1.,0.95)

/media/hpc2_storage/nvaulin/test_ties32/turbulence/turbulence0 14 ties=2309 motion=0
/media/hpc2_storage/nvaulin/test_ties32/thermal/corridor 18 ties=1239 motion=3143
/media/hpc2_storage/nvaulin/test_ties32/PTZ/continuousPan 3 ties=660 motion=759
/media/hpc2_storage/nvaulin/test_ties32/dynamicBackground/boats 22 ties=676 motion=0
/media/hpc2_storage/nvaulin/test_ties32/shadow/backdoor 5 ties=301 motion=62
/media/hpc2_storage/nvaulin/test_ties32/nightVideos/bridgeEntry 4 ties=62 motion=0
/media/hpc2_storage/nvaulin/test_ties32/intermittentObjectMotion/abandonedBox 7 ties=16 motion=0
/media/hpc2_storage/nvaulin/test_ties32/baseline/highway 3 ties=219 motion=498
/media/hpc2_storage/nvaulin/test_ties32/cameraJitter/badminton 0 ties=298 motion=296
/media/hpc2_storage/nvaulin/test_ties32/lowFramerate/port_0_17fps 6 ties=363 motion=0
/media/hpc2_storage/nvaulin/test_ties32/badWeather/blizzard 22 ties=3599 motion=3927


In [16]:
import shutil
def purge(dir):
    for i in os.listdir(dir):
        for j in os.listdir(dir+'/'+i):
            p = dir+'/'+i+'/'+j
            l = [k[:-4] for k in os.listdir(p)]
            if( not('0_motion' in l)):
                print(p,'no motion ',len(l))
                shutil.rmtree(p)
            else:
                print(p,len(l))
purge('/media/hpc2_storage/nvaulin/ties32')        
purge('/media/hpc2_storage/nvaulin/test_ties32')           

('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence2', 'no motion ', 360)
('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence3', 683)
('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence1', 661)
('/media/hpc2_storage/nvaulin/ties32/thermal/lakeSide', 139)
('/media/hpc2_storage/nvaulin/ties32/thermal/park', 199)
('/media/hpc2_storage/nvaulin/ties32/thermal/diningRoom', 225)
('/media/hpc2_storage/nvaulin/ties32/thermal/library', 524)
('/media/hpc2_storage/nvaulin/ties32/PTZ/zoomInZoomOut', 'no motion ', 136)
('/media/hpc2_storage/nvaulin/ties32/PTZ/twoPositionPTZCam', 123)
('/media/hpc2_storage/nvaulin/ties32/PTZ/intermittentPan', 384)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/overpass', 136)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/fall', 898)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/canoe', 'no motion ', 102)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/fountain02', 'no motion ', 234)
('/media/hpc2_storage/nvaulin/t